[View in Colaboratory](https://colab.research.google.com/github/youqad/Neurorobotics_Project/blob/master/Inferring_Space_from_Sensorimotor_Dependencies.ipynb)

# Inferring Space from Sensorimotor Dependencies


## The Algorithm

1. Proprioceptive input is separated from exteroceptive input by noting that proprioceptive input remains silent when no motor commands are given, whereas exteroceptive input changes because of environmental change.

2. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the environment changes. The algorithm issues no motor commands and calculates the covariance matrix of the observed environment-induced variations in sensory inputs. The dimension estimation is done by considering the eigenvalues of this covariance matrix. The eigenvalues $λ_i$ should fall into two classes: a class with values all equal to zero and a class with nonzero values. The two classes are separated by a clustering method (e.g. PCA). The number of nonzero eigenvalues is taken as the number of dimensions.

3. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the body moved. The environment is kept fixed, and the algorithm gives random motor commands. The covariance matrix of the resulting changes is observed and the dimension is estimated from the number of nonzero eigenvalues in the same way as before.

4. Estimation of the number of parameters needed to describe the changes in exteroceptive inputs when both the body and the environment change. The environment is changed at random, and the organism gives random motor commands. The number of nonzero eigenvalues of the covariance matrix is obtained as before.

## Simulation

### Notations

Notation|Meaning
-|-
$$Q ≝ (Q_1, \ldots, Q_{3q})$$|positions of the joints
$$P ≝ (P_1, \ldots, P_{3p})$$|positions of the eyes
$$a^θ_i, a^φ_i, a^ψ_i$$|Euler angles for the orientation of eye i
$$Rot(a^θ_i, a^φ_i, a^ψ_i)$$|rotation matrix for eye i
$$C_{i,k}$$|relative position of photosensor $k$ within eye $i$
$$d ≝ (d_1, \ldots,d_p)$$|apertures of diaphragms
$$L ≝ (L_1,\ldots,L_{3r})$$|positions of the lights
$$θ ≝ (θ_1, \ldots, θ_r)$$|luminances of the lights
$$S^e_{i,k}$$|sensory input from exteroceptive sensor $k$ of eye $i$
$$S^p_i$$|sensory input from proprioceptive sensor $i$
$$M, E$$|motor command and environmental control vector


### Computing the sensory inputs


$$
\begin{align*}
(Q,P,a) &≝ σ(W_1 · σ(W_2 · M − μ_2)−μ_1)\\
L &≝ σ(V_1 ·σ(V_2 · E − ν_2) − ν_1)\\
∀1≤ k ≤ p', 1≤i≤p, \quad S^e_{i,k} &≝ d_i \sum\limits_{ j } \frac{θ_j}{\Vert P_i + Rot(a_i^θ, a_i^φ, a_i^ψ) \cdot C_{i,k} - L_j \Vert^2}\\
(S^p_i)_{1≤ i ≤ q'q} &≝ σ(U_1 · σ(U_2 · Q − τ_2) − τ_1)
\end{align*}
$$

where

- $W_1, W_2, V_1, V_2, U_1, U_2$ are matrices with coefficients drawn randomly from a uniform distribution between $−1$ and $1$
- the vectors $μ_1, μ_2, ν_1, ν_2, τ_1, τ_2$ too
- $σ$ is an arbitrary nonlinearity (e.g. the hyperbolic tangent function)
- the $C_{i,k}$ are drawn from a centered normal distribution whose variance (which can be understood as the size of the retina) is so that the sensory changes resulting from a rotation of the eye are of the same order of magnitude as the ones resulting from a translation of the eye
- $θ$ and $d$ are constants drawn at random in the interval $[0.5, 1]$


In [34]:
# /!\ We're using Python 3

import plotly
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import scipy.io as sio
from scipy import stats
from scipy import special

%matplotlib inline

plotly.offline.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML, Markdown
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

# Colorscales
def colorscale_list(cmap, number_colors, return_rgb_only=False):
    cm = plt.get_cmap(cmap)
    colors = [np.array(cm(i/number_colors)) for i in range(1, number_colors+1)]
    rgb_colors_plotly = []
    rgb_colors_only = []
    for i, c in enumerate(colors):
        col = 'rgb{}'.format(tuple(255*c[:-1]))
        rgb_colors_only.append(col)
        rgb_colors_plotly.append([i/number_colors, col])
        rgb_colors_plotly.append([(i+1)/number_colors, col])
    return rgb_colors_only if return_rgb_only else rgb_colors_plotly

from scipy.io import loadmat, whosmat
from numpy.random import randint

def formatted(f): 
    return format(f, '.2f').rstrip('0').rstrip('.')

## Organism 1

**Parameter**|**Value**
-|-
Dimension of motor commands|$40$
Dimension of environmental control vector|$40$
Dimension of proprioceptive inputs|$16 \quad (= 4×4)$
Dimension of exteroceptive inputs|$40 \quad (= 2 × 20)$
Number of eyes|$2$
Number of joints|$4$
Diaphragms|None
Number of lights|$3$
Light luminance|Fixed


1. The arm has

    - $4$ joints

        - each of which has $4$ **proprioceptive** sensors (whose output depended on the position of the joint)
    - $2$ eyes (for each of them: $3$ spatial and $3$ orientation coordinates)

        - on which there are $20$ omnidirectionally sensitive photosensors (**exteroceptive**)
2. the motor command is $40$-dimensional  

3. the environment consists of:

    - $3$ lights ($3$ spatial coordinates and $3$ luminance values for each of them)

In [0]:
# Rotation matrix computation

def Rot(euler_angles):
  R = np.empty((3,3))
  c = np.cos(euler_angles)
  s = np.sin(euler_angles)

  # cf. https://en.wikipedia.org/wiki/Euler_angles#Rotation_matrix 
  R[0,0] = c[0]*c[2] - c[1]*s[0]*s[2]
  R[0,1] = -c[0]*s[2] - c[1]*c[2]*s[0]
  R[0,2] = s[0]*s[1]

  R[1,0] = s[0]*c[2] + c[0]*c[1]*s[2]
  R[1,1] = c[0]*c[1]*c[2] - s[0]*s[2]
  R[1,2] = -c[0]*s[1]

  R[2,0] = s[1]*s[2]
  R[2,1] = -c[2]*s[1]
  R[2,2] = c[1]

  return R

def computing_sensory_inputs(M, E):
  Q, P, a = [arr.reshape([-1, 3]) 
             for arr in np.split(sigma(W_1.dot(sigma(W_2.dot(M)-mu_2)) - mu_1), 
                                 [3*nb_joints, 3*nb_joints+3*nb_eyes])]
  L = sigma(V_1.dot(sigma(V_2.dot(E)-nu_2)) - nu_1).reshape([-1, 3])
  Sp = sigma(U_1.dot(sigma(U_2.dot(Q.flatten())-tau_2)) - tau_1)
  Se = np.array([d[i]*
                 sum(theta[j]/np.linalg.norm(P[i]+Rot(a[i]).dot(C[i,k])-L[j])**2
                     for j in range(nb_lights))
                 for i in range(nb_eyes)
                 for k in range(extero)])
  return np.concatenate((Sp, Se))
  
def neighborhood_lin_approx(size, neighborhood_size=neighborhood_size):
  rand_vect = np.random.normal(0, neighborhood_size, size)
  rand_vect[np.abs(rand_vect) > neighborhood_size] = 0
  return rand_vect

def PCA(data):
  cov_matrix = np.cov(data.T)
  eig_val, eig_vect = np.linalg.eigh(cov_matrix)
  
  #print(cov_matrix)
  #print([eig_val[i+1]/eig_val[i] for i in range(len(eig_val)-1)])
  
  max_ratio = np.argmax([eig_val[i+1]/eig_val[i] for i in range(len(eig_val)-1)])
  
  return len(eig_val[max_ratio+1:])
  

In [0]:
#@title Parameters

M_size = 40 #@param {type:"slider", min:35, max:45, step:1}
E_size = 40 #@param {type:"slider", min:35, max:45, step:1}

# Number of Joints / q
nb_joints = 4 #@param {type:"slider", min:3, max:5, step:1}

# Number of eyes / p
nb_eyes = 2 #@param {type:"slider", min:2, max:5, step:1}

# Number of lights / r
nb_lights = 3 #@param {type:"slider", min:2, max:5, step:1}

# Number of exteroceptive photosensors / p'
extero = 20 #@param {type:"slider", min:15, max:25, step:1}

# Number of proprioceptive sensors / q'
proprio = 4 #@param {type:"slider", min:3, max:5, step:1}


# Sensory inputs were generated from...
nb_generating_motor_commands = 40 #@param {type:"slider", min:10, max:100, step:10}
nb_generating_env_positions = 40 #@param {type:"slider", min:10, max:100, step:10}

# Neighborhood size of the linear approximation:
# Motor commands/Environmental positions drawn from normal distribution
# with mean zero and standard deviation... 
neighborhood_size = 1e-8
# (Coordinates differing from 0 by more than the std deviation are set equal to 0)

sigma = np.tanh

In [0]:
def simulation(seed):
  np.random.seed(seed)

  # Random initializations

  dim_mu1 = 3*nb_joints+3*nb_eyes+3*nb_eyes
  W_1, mu_1 = 2*np.random.rand(dim_mu1, dim_mu1)-1, 2*np.random.rand(dim_mu1)-1
  W_2, mu_2 = 2*np.random.rand(dim_mu1, M_size)-1, 2*np.random.rand(dim_mu1)-1

  dim_nu1 = 3*nb_lights
  V_1, nu_1 = 2*np.random.rand(dim_nu1, dim_nu1)-1, 2*np.random.rand(dim_nu1)-1
  V_2, nu_2 = 2*np.random.rand(dim_nu1, E_size)-1, 2*np.random.rand(dim_nu1)-1

  dim_tau1 = proprio*nb_joints
  U_1, tau_1 = 2*np.random.rand(dim_tau1, dim_tau1)-1, 2*np.random.rand(dim_tau1)-1
  U_2, tau_2 = 2*np.random.rand(dim_tau1, 3*nb_joints)-1, 2*np.random.rand(dim_tau1)-1

  theta, d = .5*np.random.rand(nb_lights)+.5, .5*np.random.rand(nb_eyes)+.5

  C = np.random.multivariate_normal(np.zeros(3), np.identity(3), (nb_eyes, extero))

  M_0, E_0 = 10*np.random.rand(M_size)-5, 10*np.random.rand(E_size)-5


  # Separating proprioceptive input from exteroceptive input
  mask_proprio = np.array([computing_sensory_inputs(M_0,
                                                    E_0+np.random.normal(0, neighborhood_size, E_size))
                  for _ in range(nb_generating_env_positions)])
  mask_proprio = np.all(mask_proprio == mask_proprio[0, :], axis=0)


  # Number of parameters needed to describe the variations 
  # in the exteroceptive inputs when 
  # 1. only the environment changes
  # 2. only the motor commands change
  # 3. both change

  env_variations = np.array([
      computing_sensory_inputs(M_0,
                               E_0+neighborhood_lin_approx(E_size))[~mask_proprio]
      for _ in range(nb_generating_env_positions)])

  mot_variations = np.array([
      computing_sensory_inputs(M_0+neighborhood_lin_approx(M_size),
                               E_0)[~mask_proprio]
      for _ in range(nb_generating_motor_commands)])

  env_mot_variations = np.array([
      computing_sensory_inputs(M_0+neighborhood_lin_approx(M_size),
                               E_0+neighborhood_lin_approx(E_size))[~mask_proprio]
      for _ in range(nb_generating_env_positions*nb_generating_motor_commands)])

  # Now PCA!

  dim_env = PCA(env_variations)
  dim_extero = PCA(mot_variations)
  dim_env_extero = PCA(env_mot_variations)

  table = '''
  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|{}
  Environment|{}
  Body and Environment simultaneous changes|{}
  Group of compensated movements|{}
  '''.format(dim_extero, dim_env, dim_env_extero,
             dim_env+dim_extero-dim_env_extero)

  return table

In [93]:
for seed in range(8):
  print(simulation(seed))


  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|8
  Environment|3
  Body and Environment simultaneous changes|11
  Group of compensated movements|0
  

  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|3
  Environment|1
  Body and Environment simultaneous changes|3
  Group of compensated movements|1
  

  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|5
  Environment|7
  Body and Environment simultaneous changes|9
  Group of compensated movements|3
  

  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|8
  Environment|3
  Body and Environment simultaneous changes|9
  Group of compensated movements|2
  

  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|9
  Environment|2
  Body and Environment simultaneous changes|7
  Group of compensated movements|4
  

  ||Number of parameters necessary to describe|
  -|-|
  Exteroceptive body|1
  Environment|2
  Body an